In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from dataloader import get_MNIST
from models import get_4_layer, get_8_layer, get_VGG16


In [ ]:
model_dict = {
    
    # 4 layer CNN
    # One Conv Layer + Max Pooling + 3 Dense Layers (softmax output)
    # Data = (n, 28, 28, 1)
    #----------------------------------------------------------------------------
    'four_layer': (get_MNIST(vgg_preprocess=False), get_4_layer(num_classes=10)),
    
    # 8 layer CNN
    # 5 Conv Layera + Max Pooling + 3 Dense Layers (softmax output)
    # Data = (n, 28, 28, 1)
    #----------------------------------------------------------------------------
    'eight_layer': (get_MNIST(vgg_preprocess=False), get_8_layer(num_classes=10)),
    
    # VGG16
    # 13 Conv Layera + Max Pooling + 3 Dense Layers (softmax output)
    # Data = (n, 48, 48, 3) - * VGG requires > 30 input features + >= 3 channels
    'vgg16': (get_MNIST(vgg_preprocess=True), get_VGG16(num_classes=10)),
    }




In [3]:
BATCH_SIZE = 256 #* as specified in paper
EPOCHS = 100 #They train for 10s of thousands of iterations?????
LR = 3e-5 #* as specified in paper
LOSS = "categorical_crossentropy"
OPTIMIZER = Adam(LR)
METRICS = ['accuracy']

for model_name in model_dict.keys():
    print(f'training model {model_name}')
    (x_train, y_train, x_test, y_test) = model_dict[model_name][0]
    model = model_dict[model_name][1]
    model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=METRICS)
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)


training model vgg16
Epoch 1/100
  6/211 [..............................] - ETA: 13:09 - loss: 2.1797 - accuracy: 0.2741

KeyboardInterrupt: 